# Sensitivity analysis of the NF-kB pathway
The steps for this are as follow:
1. From an equations text file, load the reaction equations and create a state space model
2. From the state space model and initial parameters, find 
    1. The initial parameters for the model (run it 1000 times from the first initial conditions)
    2. Determine the value of x<sub>i</sub> for all i for all t in [0, N]
3. Calculate s<sub>i, j</sub> for all t given the values of x<sub>i</sub> for all t
4. Calcuate sbar for all t
5. Having the sensitivity matrix and the values of x for all desired t, calculate the sensitivity equations

We need x<sub>i</sub>(t) and s<sub>i, j</sub> to calculate sbar, which is needed for sensitivity analysis. Sensitivity analysis for all constants is defined as 
![Equation 22](./equation22.png)
where s bar is
![s bar](./sbar.png)
dx/dtheta can be found from S
![S](./bigS.png)
and an entry  s<sub>i, j</sub> is 
![s](./littleS.png)
where x<sub>i</sub> is the integration of the system of equations from the state space model and delta is the Kronecker delta function 
![kronecker delta](./kroneckerDelta.png)
### All necessary imports

In [1]:
import numpy as np
import sympy as sp
from scipy.integrate import odeint

### All program constants

In [2]:
initial_conditions_N = 2000
N = 400

pathway_file = 'pathway_eqs.txt'
reaction_coefficients_file = 'reaction_coefficients.txt'

### Load the constants and separate them 

In [3]:
ks = {}
with open(reaction_coefficients_file, 'r') as o: 
    for l in o:
        k = l.split('=')[0]
        ks[k] = float(eval(l.split('=')[1].replace('\n', '').strip()))
print(ks)

{'k1': 0.5, 'k2': 0.0005, 'k3': 0.5, 'k4': 0.0005, 'k5': 0.5, 'k6': 0.0005, 'k7': 0.5, 'k8': 0.0005, 'k9': 0.0204, 'k10': 0.5, 'k11': 0.0005, 'k12': 0.0075, 'k13': 0.5, 'k14': 0.0005, 'k15': 0.011000000000000001, 'k16': 2.25e-05, 'k17': 2.25e-05, 'k18': 2.25e-05, 'k19': 0.09000000000000001, 'k20': 8e-05, 'k21': 0.5, 'k22': 0.0005, 'k23': 0.5, 'k24': 0.0005, 'k25': 0.5, 'k26': 0.0005, 'k27': 1.5399999999999999e-06, 'k28': 0.0165, 'k29': 0.00028, 'k30': 1.78e-07, 'k31': 0.00028, 'k32': 1.2699999999999999e-07, 'k33': 0.00028, 'k34': 0.0225, 'k35': 0.00125, 'k36': 0.00408, 'k37': 0.000113, 'k38': 0.00030000000000000003, 'k39': 0.0002, 'k40': 0.006, 'k41': 0.00175, 'k42': 0.00408, 'k43': 0.000113, 'k44': 0.00015000000000000001, 'k45': 0.0001, 'k46': 0.009000000000000001, 'k47': 0.00175, 'k48': 0.00408, 'k49': 0.000113, 'k50': 0.00015000000000000001, 'k51': 0.0001, 'k52': 0.18500000000000003, 'k53': 0.00125, 'k54': 0.0138, 'k55': 0.048, 'k56': 0.00175, 'k57': 0.005200000000000001, 'k58': 0.0

## 1. Generate the state space model from reaction equations

In [4]:
import numpy as np

file = open(pathway_file,"r")
eqs_arr=file.readlines()
file.close()
eqs_arr=[i[:-1] if i[-1] == '\n' else i for i in eqs_arr ] #remove \n

class Rxn:
    
    def __init__(self, ID, reactants, products):
        #reactants and products are lists
        self.ID = str(ID)
        self.reactants = reactants
        self.products = products
        
    def __str__(self):
        ret = "ID: " + self.ID + " Reactants: "
        for r in self.reactants:
            ret = ret + r.ID + ", "
        ret = ret + "Products: "
        for p in self.products:
            ret = ret + p.ID + ", "
        return ret

class RxnSpec:
    
    def __init__(self, ID, name):
        self.ID = 'x' + str(ID)
        self.name = name
        
    def __str__(self):
        return 'ID: ' + self.ID + ' name: ' + self.name

#list of rxn species
rxn_species = [RxnSpec(1,'ikba'), RxnSpec(2,'nf-kb'), RxnSpec(3,'ikba-nf-kb'), RxnSpec(4,'ikbb'), RxnSpec(5,'ikbb-nf-kb'),
              RxnSpec(6,'ikbe'),RxnSpec(7,'ikbe-nf-kb'),RxnSpec(8,'ikkikba'),RxnSpec(9,'ikkikba-nf-kb'),RxnSpec(10,'ikk'),
              RxnSpec(11,'ikkikbb'),RxnSpec(12,'ikkikbb-nf-kb'),RxnSpec(13,'ikkikbe'),RxnSpec(14,'ikkikbe-nf-kb'),
              RxnSpec(15,'nf-kb_n'),RxnSpec(16,'ikba_n'),RxnSpec(17,'ikba_n-nf-kb_n'),RxnSpec(18,'ikbb_n'),RxnSpec(19,'ikbb_n-nf-kb_n'),
              RxnSpec(20,'ikbe_n'),RxnSpec(21,'ikbe_n-nf-kb_n'),RxnSpec(22,'ikba_-t'),RxnSpec(23,'ikbb_-t'),RxnSpec(24,'ikbe_-t'),RxnSpec('SOURCE','source'),RxnSpec('SINK','sink')]

rxn_arr = []

for idx,eq in enumerate(eqs_arr):
    temp = eq.split('->')
    reactants = temp[0].split('+')
    products = temp[1].split('+')
    #convert each element in reactants and products into a RxnSpec
    new_reactants = []
    new_products = []
    for reac in reactants:
        for rs in rxn_species:
            if rs.name == reac:
                new_reactants.append(rs)
                break
    for prod in products:
        for rs in rxn_species:
            
            if rs.name == prod:
                new_products.append(rs)
                break
    
    
    rxn = Rxn(idx+1,new_reactants, new_products)
    rxn_arr.append(rxn)


for rxn in rxn_arr:
    print(rxn)

state_space_model = []

for rs in rxn_species:
    eq = rs.ID + '(t) = '
    for rxn in rxn_arr:
        if rs.name in [i.name for i in rxn.reactants]:
            k = '-k' + str(rxn.ID)
            eq = eq + k
            for reac in rxn.reactants:
                eq = eq + '*' + reac.ID
            
        if rs.name in [i.name for i in rxn.products]:
            k = '+k' + str(rxn.ID)
            eq = eq + k
            for reac in rxn.reactants:
                eq = eq + '*' + reac.ID
    state_space_model.append(eq)
    
for eq in state_space_model:
    print(eq)

ID: 1 Reactants: x1, x2, Products: x3, 
ID: 2 Reactants: x3, Products: x2, x1, 
ID: 3 Reactants: x4, x2, Products: x5, 
ID: 4 Reactants: x5, Products: x2, x4, 
ID: 5 Reactants: x6, x2, Products: x7, 
ID: 6 Reactants: x7, Products: x2, x6, 
ID: 7 Reactants: x8, x2, Products: x9, 
ID: 8 Reactants: x9, Products: x2, x8, 
ID: 9 Reactants: x9, Products: x10, x2, 
ID: 10 Reactants: x11, x2, Products: x12, 
ID: 11 Reactants: x12, Products: x2, x11, 
ID: 12 Reactants: x12, Products: x10, x2, 
ID: 13 Reactants: x13, x2, Products: x14, 
ID: 14 Reactants: x14, Products: x2, x13, 
ID: 15 Reactants: x14, Products: x10, x2, 
ID: 16 Reactants: x3, Products: x2, 
ID: 17 Reactants: x5, Products: x2, 
ID: 18 Reactants: x7, Products: x2, 
ID: 19 Reactants: x2, Products: x15, 
ID: 20 Reactants: x15, Products: x2, 
ID: 21 Reactants: x16, x15, Products: x17, 
ID: 22 Reactants: x17, Products: x15, x16, 
ID: 23 Reactants: x18, x15, Products: x19, 
ID: 24 Reactants: x19, Products: x15, x18, 
ID: 25 Reactants: 

## 2. Identify the initial conditions and x(t) for all t
1. Run the integration initial_conditions_N number of times to get initial conditions
2. Augment these by any thing you need from the paper for actual initial conditions

__NOTE__:   as of now, hard code the k values (from some source) and the partial derivatives (from the state space model)

In [5]:
# followed the direction of this video https://www.youtube.com/watch?v=zRMmiBMjP9o
def nf_kb(x, t):
    # x is here to be used by (1) initial conditions and (2) x when we use odeint
    # t is here for obvious reasons
    
    # define the k values
    k1=0.5
    k2=0.5*10**-3 
    k3=0.5
    k4=0.5*10**-3 
    k5=0.5
    k6=0.5*10**-3 
    k7=0.5
    k8=0.5*10**-3 
    k9=2.04*10**-2 
    k10=0.5
    k11=0.5*10**-3 
    k12=7.5*10**-3 
    k13=0.5
    k14=0.5*10**-3 
    k15=1.1*10**-2 
    k16=2.25*10**-5 
    k17=2.25*10**-5 
    k18=2.25*10**-5 
    k19=0.9*10**-1 
    k20=0.8*10**-4 
    k21=0.5
    k22=0.5*10**-3 
    k23=0.5
    k24=0.5*10**-3 
    k25=0.5
    k26=0.5*10**-3 
    k27=1.54*10**-6  
    k28=1.65*10**-2  
    k29=2.8*10**-4 
    k30=1.78*10**-7  
    k31=2.8*10**-4 
    k32=1.27*10**-7  
    k33=2.8*10**-4 
    k34=22.5*10**-3  
    k35=1.25*10**-3 
    k36=4.08*10**-3 
    k37=1.13*10**-4 
    k38=3*10**-4 
    k39=2*10**-4 
    k40=6.0*10**-3  
    k41=1.75*10**-3 
    k42=4.08*10**-3 
    k43=1.13*10**-4 
    k44=1.5*10**-4 
    k45=1*10**-4 
    k46=9.0*10**-3  
    k47=1.75*10**-3 
    k48=4.08*10**-3 
    k49=1.13*10**-4 
    k50=1.5*10**-4 
    k51=1*10**-4 
    k52=1.85*10**-1  
    k53=1.25*10**-3 
    k54=1.38*10**-2 
    k55=4.8*10**-2  
    k56=1.75*10**-3 
    k57=5.2*10**-3 
    k58=7.0*10**-2  
    k59=1.75*10**-3 
    k60=5.2*10**-3 
    k61=1.2*10**-4 
    k62=4.07*10**-3 
    k63=1.5*10**-3 
    k64=2.2*10**-3 
    
    # im lazy and make it easy to read
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    x4 = x[3]
    x5 = x[4]
    x6 = x[5]
    x7 = x[6]
    x8 = x[7]
    x9 = x[8]
    x10 = x[9]
    x11 = x[10]
    x12 = x[11]
    x13 = x[12]
    x14 = x[13]
    x15 = x[14]
    x16 = x[15]
    x17 = x[16]
    x18 = x[17]
    x19 = x[18]
    x20 = x[19]
    x21 = x[20]
    x22 = x[21]
    x23 = x[22]
    x24 = x[23]
    
    dx1dt = -k1*x1*x2+k2*x3-k34*x10*x1+k35*x8+k36*x22-k37*x1-k38*x1+k39*x16
    dx2dt = -k1*x1*x2+k2*x3-k3*x4*x2+k4*x5-k5*x6*x2+k6*x7-k7*x8*x2+k8*x9+k9*x9-k10*x11*x2+k11*x12+k12*x12-k13*x13*x2+k14*x14+k15*x14+k16*x3+k17*x5+k18*x7-k19*x2+k20*x15
    dx3dt = +k1*x1*x2-k2*x3-k16*x3-k52*x10*x3+k53*x9+k54*x17
    dx4dt = -k3*x4*x2+k4*x5-k40*x10*x4+k41*x11+k42*x23-k43*x4-k44*x4+k45*x18
    dx5dt = +k3*x4*x2-k4*x5-k17*x5-k55*x10*x5+k56*x12+k57*x19
    dx6dt = -k5*x6*x2+k6*x7-k46*x10*x6+k47*x13+k48*x24-k49*x6-k50*x6+k51*x20
    dx7dt = +k5*x6*x2-k6*x7-k18*x7-k58*x10*x7+k59*x14+k60*x21
    dx8dt = -k7*x8*x2+k8*x9+k34*x10*x1-k35*x8-k62*x8
    dx9dt = +k7*x8*x2-k8*x9-k9*x9+k52*x10*x3-k53*x9
    dx10dt = +k9*x9+k12*x12+k15*x14-k34*x10*x1+k35*x8-k40*x10*x4+k41*x11-k46*x10*x6+k47*x13-k52*x10*x3+k53*x9-k55*x10*x5+k56*x12-k58*x10*x7+k59*x14-k61*x10+k62*x8+k63*x11+k64*x13
    dx11dt = -k10*x11*x2+k11*x12+k40*x10*x4-k41*x11-k63*x11
    dx12dt = +k10*x11*x2-k11*x12-k12*x12+k55*x10*x5-k56*x12
    dx13dt = -k13*x13*x2+k14*x14+k46*x10*x6-k47*x13-k64*x13
    dx14dt = +k13*x13*x2-k14*x14-k15*x14+k58*x10*x7-k59*x14
    dx15dt = +k19*x2-k20*x15-k21*x16*x15+k22*x17-k23*x18*x15+k24*x19-k25*x20*x15+k26*x21-k28*x15*x15+k28*x15*x15
    dx16dt = -k21*x16*x15+k22*x17+k38*x1-k39*x16
    dx17dt = +k21*x16*x15-k22*x17-k54*x17
    dx18dt = -k23*x18*x15+k24*x19+k44*x4-k45*x18
    dx19dt = +k23*x18*x15-k24*x19-k57*x19
    dx20dt = -k25*x20*x15+k26*x21+k50*x6-k51*x20
    dx21dt = +k25*x20*x15-k26*x21-k60*x21
    dx22dt = +k27+k28*x15*x15-k29*x22-k36*x22+k36*x22
    dx23dt = +k30-k31*x23-k42*x23+k42*x23
    dx24dt = +k32-k33*x24-k48*x24+k48*x24
    
    
    return [dx1dt, dx2dt, dx3dt, dx4dt, dx5dt, dx6dt, dx7dt, dx8dt,
            dx9dt, dx10dt, dx11dt, dx12dt, dx13dt, dx14dt, dx15dt, dx16dt, 
            dx17dt, dx18dt, dx19dt, dx20dt, dx21dt, dx22dt, dx23dt, dx24dt]
# first run for the initial conditions
T = initial_conditions_N
x0x0 = [0, 0.1] + [0 for i in range(22)]
t = np.asarray(range(T))

init_conditions = odeint(nf_kb, x0x0, t)


Now we have the initial conditions, so we need to augment this by paper specific parameters

In [10]:
# ok now do it for the simulation
T = N
x0 = init_conditions[-1]
# fill in the extra stuff in the paper, IKK at 0.1 (x10, index 9) PAPER SPECIFIC
x0[9] = 0.1
t = np.asarray(range(T))

x = odeint(nf_kb, x0, t)
# x has the shape of 
# [N, i] (an entry for all x for all N)
# replace x[0] with the initial conditions
x[0] = x0

[5.96447746e-01 1.45866095e-04 9.50619356e-02 1.00460845e-03
 7.84199527e-05 7.16559797e-04 5.58304348e-05 1.31932135e-03
 1.72781464e-03 9.69390702e-02 5.92487908e-07 3.69528749e-07
 6.33775801e-07 3.83403409e-07 7.59981153e-04 4.91616450e-02
 2.15752159e-03 4.03767590e-05 6.93186434e-06 2.88081211e-05
 4.94576838e-06 1.17023585e-01 2.72588523e-04 1.94487317e-04]


## 3. Calculate s for all t
The equation for s<sub>i, j</sub> is
![s](./littleS.png)
where delta is 
![kronecker delta](./kroneckerDelta.png)
We need this s because S is defined as
![S](./bigS.png)
which we need for the calculation of sbar

In [12]:
# do the delta function
def kdf(i, j, xt, theta):
    if (i != j):
        return 0
    return theta - xt

In [13]:
s = []
# has the shape (N, i, j) N number time steps, i range of x, j range of theta
for k in range(N): # go through all timesteps of the sim
    st = [[0 for _ in range(len(ks))] for _ in range(len(x[0]))]
    for i in range(len(x[0])): # go through all of the xs
        for j in range(len(ks)): # go through all of the js
            kkey = 'k{}'.format(j + 1)
            st[i][j] = kdf(i, j, x[k][i], ks[kkey])
            s.append(st)

## 4. S bar
s bar is defined as:
![s bar](./sbar.png)

In [14]:
# find sbar for all t
def sbar(i, j, k):
    return s[k][i][j] * ks['k{}'.format(j+1)] / x[k][i]

## 5 the sensitivity constants 

### Dynamic sensitivity analysis

#### Dynamic sensitivity analysis with multiple variables
This is equation 22 of the paper
![Equation 22](./equation22.png)

In [15]:
# do the innermost summation
import math
def inner_sum(k, j, i_range):
    tosum = []
    for i in range(i_range):
        added = sbar(i, j, k)
        if math.isnan(added):
            print(i, j, k)
        tosum.append(sbar(i, j, k) ** 2)
    return sum(tosum)

In [16]:
# do the outer summation
def outer_sum(N, j, i_range):
    tosum = []
    for k in range(N):
        tosum.append(inner_sum(k, j, i_range))
    return sum(tosum)

In [17]:
# do the 1/n and the sqrt
def eqn22(N, j, i_range):
    return (1/N) * np.sqrt(outer_sum(N, j, i_range))

In [18]:
# do the sensitivity analysis
i_range = len(x[0])
j_range = len(ks.keys())
OS = []
for j in range(j_range):
    print('Calculating sensitivity for j {}/{}\r'.format(j + 1, j_range), end='')
    OS.append(eqn22(N, j, i_range))
    

7 0 0
8 0 0
10 0 0
11 0 0
12 0 0
13 0 0
7 1 0
8 1 0
10 1 0
11 1 0
12 1 0
13 1 0
7 2 0
8 2 0
10 2 0
11 2 0
12 2 0
13 2 0
7 3 0
8 3 0
10 3 0
11 3 0
12 3 0
13 3 0
7 4 0
8 4 0
10 4 0
11 4 0
12 4 0
13 4 0
7 5 0
8 5 0
10 5 0
11 5 0
12 5 0
13 5 0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


7 6 0
8 6 0
10 6 0
11 6 0
12 6 0
13 6 0
8 7 0
10 7 0
11 7 0
12 7 0
13 7 0
7 8 0
10 8 0
11 8 0
12 8 0
13 8 0
7 9 0
8 9 0
10 9 0
11 9 0
12 9 0
13 9 0
7 10 0
8 10 0
11 10 0
12 10 0
13 10 0
7 11 0
8 11 0
10 11 0
12 11 0
13 11 0
7 12 0
8 12 0
10 12 0
11 12 0
13 12 0
7 13 0
8

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


 13 0
10 13 0
11 13 0
12 13 0
7 14 0ating sensitivity for j 15/64
8 14 0
10 14 0
11 14 0
12 14 0
13 14 0
7 15 0ating sensitivity for j 16/64
8 15 0
10 15 0
11 15 0
12 15 0
13 15 0
7 16 0ating sensitivity for j 17/64
8 16 0
10 16 0
11 16 0
12 16 0
13 16 0
7 17 0ating sensitivity for j 18/64
8 17 0
10 17 0
11 17 0
12 17 0
13 17 0
7 18 0ating sensitivity for j 19/64
8 18 0
10 18 0
11 18 0
12 18 0
13 18 0
7 19 0ating sensitivity for j 20/64
8 19 0
10 19 0
11 19 0
12 19 0
13 19 0
7 20 0ating sensitivity for j 21/64
8 20 0
10 20 0
11 20 0
12 20 0
13 20 0
7 21 0ating sensitivity for j 22/64
8 21 0
10 21 0
11 21 0
12 21 0
13 21 0
7 22 0ating sensitivity for j 23/64
8 22 0
10 22 0
11 22 0
12 22 0
13 22 0
7 23 0ating sensitivity for j 24/64
8 23 0
10 23 0
11 23 0
12 23 0
13 23 0
7 24 0ating sensitivity for j 25/64
8 24 0
10 24 0
11 24 0
12 24 0
13 24 0
7 25 0ating sensitivity for j 26/64
8 25 0
10 25 0
11 25 0
12 25 0
13 25 0
7 26 0ating sensitivity for j 27/64
8 26 0
10 26 0
11 26 0
12 26 0
13 

In [20]:
print(x[0])

[5.97521108e-01 1.30181698e-04 9.67877617e-02 1.00437832e-03
 7.87277938e-05 7.16607007e-04 5.61709540e-05 0.00000000e+00
 0.00000000e+00 1.00000000e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 7.65494124e-04 4.90100028e-02
 2.16974463e-03 4.02420143e-05 6.95607467e-06 2.87119990e-05
 4.96304204e-06 1.17054807e-01 2.72486833e-04 1.94414763e-04]


In [19]:
print(OS)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [37]:
np.argmax(OS)

20

In [38]:
counted = [(i, OS[i]) for i in range(len(OS))]

In [39]:
counted.sort(key=lambda x: x[1], reverse=True)

In [40]:
print(counted)

[(20, 0.016657963600735237), (22, 0.015458556500973672), (24, 0.015457869378558114), (51, 0.012087672319270507), (37, 0.0114716982183754), (33, 0.011201668109221151), (54, 0.01097298612841659), (57, 0.010967998694573607), (0, 0.010378925802877643), (43, 0.010104939354062186), (49, 0.01010320717541259), (45, 0.009799252518173743), (39, 0.009792770181718143), (4, 0.009052540055358499), (2, 0.009039992405808885), (35, 0.008386068060703527), (47, 0.006599622903738143), (41, 0.006597011610317031), (18, 0.005344314232942063), (27, 0.004031562388613672), (19, 0.0037508731839281796), (29, 0.003205739969274607), (31, 0.0032057399692746063), (26, 0.002088712193540339), (6, 0.001403286821436492), (12, 0.0013503346526563612), (9, 0.001315627732745468), (8, 0.0013047716957365351), (53, 0.0011703880064637261), (14, 0.0005500000298976577), (59, 0.0003956258634580093), (11, 0.0003750000350854319), (56, 0.00036825258154897983), (61, 0.00020358671577004522), (63, 0.00011000000000853507), (58, 9.14831313